In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

glueContext = GlueContext(SparkContext.getOrCreate())

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
8,application_1632897281066_0009,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
salesDF = glueContext.create_dynamic_frame.from_catalog(
             database="johndb",
             table_name="sales")
customerDF = glueContext.create_dynamic_frame.from_catalog(
             database="johndb",
             table_name="customers")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
salesDF.toDF().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+---------------+---------+---------------+-------+---------------+-------+------+--------+-------+-----------+----+-----------+--------+----------+
|ordernumber|quantityordered|priceeach|orderlinenumber|  sales|      orderdate| status|qtr_id|month_id|year_id|productline|msrp|productcode|dealsize|customerid|
+-----------+---------------+---------+---------------+-------+---------------+-------+------+--------+-------+-----------+----+-----------+--------+----------+
|      10107|             30|     95.7|              2| 2871.0| 2/24/2003 0:00|Shipped|     1|       2|   2003|Motorcycles|  95|   S10_1678|   Small|         1|
|      10121|             34|    81.35|              5| 2765.9|  5/7/2003 0:00|Shipped|     2|       5|   2003|Motorcycles|  95|   S10_1678|   Small|         2|
|      10134|             41|    94.74|              2|3884.34|  7/1/2003 0:00|Shipped|     3|       7|   2003|Motorcycles|  95|   S10_1678|  Medium|         3|
|      10145|             45|    8

In [6]:
salesDF.printSchema()
customerDF.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- ordernumber: long
|-- quantityordered: long
|-- priceeach: double
|-- orderlinenumber: long
|-- sales: double
|-- orderdate: string
|-- status: string
|-- qtr_id: long
|-- month_id: long
|-- year_id: long
|-- productline: string
|-- msrp: long
|-- productcode: string
|-- dealsize: string
|-- customerid: long

root
|-- customerid: long
|-- customername: string
|-- email: string
|-- city: string
|-- country: string
|-- territory: string
|-- contactfirstname: string
|-- contactlastname: string

In [7]:
customersalesDF=Join.apply(salesDF, customerDF, 'customerid', 'customerid')
customersalesDF.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- orderlinenumber: long
|-- sales: double
|-- territory: string
|-- city: string
|-- customername: string
|-- year_id: long
|-- productline: string
|-- contactlastname: string
|-- country: string
|-- .customerid: long
|-- qtr_id: long
|-- msrp: long
|-- quantityordered: long
|-- productcode: string
|-- status: string
|-- email: string
|-- contactfirstname: string
|-- ordernumber: long
|-- orderdate: string
|-- customerid: long
|-- dealsize: string
|-- month_id: long
|-- priceeach: double

In [8]:
customersalesDF = customersalesDF.drop_fields(['`.customerid`'])
customersalesDF.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- orderlinenumber: long
|-- sales: double
|-- territory: string
|-- city: string
|-- customername: string
|-- year_id: long
|-- productline: string
|-- contactlastname: string
|-- country: string
|-- qtr_id: long
|-- msrp: long
|-- quantityordered: long
|-- productcode: string
|-- status: string
|-- email: string
|-- contactfirstname: string
|-- ordernumber: long
|-- orderdate: string
|-- customerid: long
|-- dealsize: string
|-- month_id: long
|-- priceeach: double

In [9]:
glueContext.write_dynamic_frame.from_options(customersalesDF, connection_type = "s3", connection_options = {"path": "s3://john-data-lake/data/customersales"}, format = "json")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…